#### Mounting Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Setting up base paths

In [ ]:
# NOT TO BE CHANGED IN SUBMISSION
BASE_INPUT_PATH = "/content/drive/MyDrive/SP24-CSEN240-2/Project"
TRAIN_DATA_PATH = BASE_INPUT_PATH + "/Train"
VALIDATION_DATA_PATH = BASE_INPUT_PATH + "/Validation"
TEST_DATA_PATH = BASE_INPUT_PATH + "/Test"

In [ ]:
GROUP_NUMBER = "4" # TODO
ADDITIONAL_FILES_PATH = BASE_INPUT_PATH + "/Temp/Group-%s" % GROUP_NUMBER
OUTPUT_PATH = BASE_INPUT_PATH + "/Prediction"

AUGMENTATION_DATA_PATH = ADDITIONAL_FILES_PATH + "/Augmentation" #point to folder where augmented data will be saved
MODEL_PATH = "/content" #point to where model will be save and read

PREDICTION_FILE_PATH = OUTPUT_PATH + "/group-4.txt"
TEST_LABEL_PATH = TEST_DATA_PATH+"/labels.txt"

#### Install dependencies

In [ ]:
!pip install opencv-python scikit-learn numpy
!pip install -U libsvm-official

  Preparing metadata (setup.py) ... done
  Created wheel for libsvm-official: filename=libsvm_official-3.32.0-cp310-cp310-linux_x86_64.whl size=123887 sha256=37360fa37d7b74ef40f9c80bdc1958ae85abe038b5f9f0ad1db736600e6edbd8
  Stored in directory: /root/.cache/pip/wheels/61/3b/1b/73bb4869517f96a26c82b47ccdb9ec48f12f4466de2371eff6
Successfully built libsvm-official


#### Import dependencies

In [ ]:
# TODO
import os
import cv2
import numpy as np
import scipy
from collections import Counter
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import classification_report, accuracy_score
from sklearn.svm import SVC
from sklearn.multiclass import OneVsOneClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from libsvm.svmutil import *
from joblib import dump, load
from sklearn.decomposition import PCA
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import array_to_img, img_to_array, load_img
from os import listdir
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from statistics import mode as statistics_mode
import random
from collections import Counter

/usr/local/lib/python3.10/dist-packages/libsvm/svm.py:147: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def csr_to_problem_jit(l, x_val, x_ind, x_rowptr, prob_val, prob_ind, prob_rowptr, indx_start):


#### Define Utility Functions

In [ ]:
def write_predictions_to_file(y_pred, path):

    with open(path, 'w') as file:
        for idx, pred in enumerate(y_pred, start=1):
            file.write(f"{idx:03d} {pred}\n")
    print(f"Predictions written to {path}")

def hard_voting(y_svm_pred, y_rf_pred, y_logreg_pred, y_knn_pred):

    final_predictions = []

    # Iterate over the predictions from each classifier
    for predictions in zip(y_svm_pred, y_rf_pred, y_logreg_pred, y_knn_pred):
        # Count the occurrences of each prediction
        counts = Counter(predictions)
        # Find the maximum occurrence
        max_count = max(counts.values())
        # Collect all predictions that have the maximum occurrence
        top_candidates = [pred for pred, count in counts.items() if count == max_count]
        # Randomly select from the top candidates in case of a tie
        vote_result = random.choice(top_candidates)
        final_predictions.append(vote_result)

    return np.array(final_predictions)


# Replace your usage of hard_voting in the script with this updated function.
#function to load images and labels
def load_images_and_labels(label_file, image_folder, target_size=(50, 50)):
    images = []
    labels = []

    # Read the label file and store the labels in a dictionary
    label_dict = {}
    with open(label_file, 'r') as file:
        for line in file:
            parts = line.strip().split()
            if len(parts) == 2:
                image_id, label = parts
                label_dict[image_id] = label

    # Load images corresponding to the labels and resize them
    for image_id, label in label_dict.items():
        image_path = os.path.join(image_folder, f"{image_id}.jpg")
        if os.path.exists(image_path):
            img = cv2.imread(image_path, cv2.IMREAD_COLOR)  # Read the image
            if img is not None:
                img_resized = cv2.resize(img, target_size)  # Resize the image
                images.append(img_resized)  # Append the resized image
                labels.append(label)
                print(f"load image {image_id}.jpg")
            else:
                print(f"Failed to load image {image_id}.jpg.")
        else:
            print(f"Image file {image_id}.jpg not found.")

    return images, labels

#function to resize images
def resize_images(images, target_size=(50, 50)):
    resized_images = []
    for img in images:
        if img is not None:
            resized_img = cv2.resize(img, target_size)  # Resize the image
            resized_images.append(resized_img)  # Append the resized image
        else:
            resized_images.append(None)  # Append None for any failed image loads
    return resized_images

#function to convert color images to grayscale
def convert_to_grayscale(images):
    grayscale_images = []
    for img in images:
        if img is not None:
            gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
            grayscale_images.append(gray_img)  # Flatten the image
        else:
            grayscale_images.append(None)  # Append None for any failed image loads
    return grayscale_images

#function to flatten a list of images
def flatten_images(image_list):
    flattened_images = []
    for img in image_list:
        if img is not None:  # Ensure the image was loaded correctly
            flattened = img.flatten()  # Flatten the image
            flattened_images.append(flattened)
        else:
            flattened_images.append(None)  # Handle cases where an image might not be loaded
    return flattened_images

#funtion to normalize pixel values to specific range
def normalize_images(images):
    normalized_images = []
    for img in images:
        if img is not None:
            normalized_img = img.astype(np.float32) / 255.0  # Normalize pixel values to [0, 1]
            normalized_images.append(normalized_img)  # Append the normalized image
        else:
            normalized_images.append(None)  # Append None for any failed image loads
    return normalized_images

#image augementation function
datagen = ImageDataGenerator(
        rotation_range = 10,
       # shear_range = 0.2,
       # zoom_range = 0.1,
        horizontal_flip = True,
        brightness_range=[0.8, 1.2],  # Adjust brightness between 80% and 120% of original
        #contrast_range=[0.8, 1.2],      # Adjust contrast between 80% and 120% of original
        width_shift_range= 0.05,  # Shift images horizontally by up to 10%
        height_shift_range= 0.05,  # Shift images vertically by up to 10%
        rescale=1./255,  # Normalize pixel values to the range [0, 1]
        #target_size=(150, 150),  # Resize images to 150x150 pixels,
        fill_mode='nearest'  # Use the nearest pixel value for filling
)

#function to load images with CascadeClassifier
def cascade_load_images_and_labels(label_file, image_folder, target_size=(50, 50)):
    images = []
    labels = []

    # Read the label file and store the labels in a dictionary
    label_dict = {}
    with open(label_file, 'r') as file:
        for line in file:
            parts = line.strip().split()
            if len(parts) == 2:
                image_id,label = parts
                label_dict[image_id] = label
    haar_cascade = cv2.CascadeClassifier(MODEL_PATH+"/haarcascade_frontalface_default.xml")
    if haar_cascade.empty():
        print("Error: Failed to load cascade classifier.")
    else:
        print("Cascade classifier loaded successfully.")
    # Load images corresponding to the labels and resize them
    for image_id, label in label_dict.items():
        image_path = os.path.join(image_folder, f"{image_id}.jpg")
        if os.path.exists(image_path):
            img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Read the image
            if img is not None:
                faces = haar_cascade.detectMultiScale(img, 1.1, 9)

                for (x, y, w, h) in faces:
                #cv2.rectangle(img, (x, y), (x+w, y+h), (0, 0, 255), 2)
                    face = img[y:y + h, x:x + w]
                if face is None or img.size == 0:
                    print("Error: Failed to load FACE.")
                else:
                    print("FACE loaded successfully.")
                #cv2.imwrite('face.jpg', faces)
                img_resized = cv2.resize(face, target_size)  # Resize the image
                images.append(img_resized)  # Append the resized image
                labels.append(label)

                print(f"load image {image_id}.jpg")
            else:
                print(f"Failed to load image {image_id}.jpg.")
        else:
            print(f"Image file {image_id}.jpg not found.")

    return images, labels

def cascade_load_images(image_folder, target_size=(50, 50)):
    images = []

    # List all jpg files in the directory and sort them numerically
    image_files = [f for f in os.listdir(image_folder) if f.endswith('.jpg')]
    image_files.sort(key=lambda x: int(x.split('.')[0]))  # Sorting numerically by file name

    haar_cascade = cv2.CascadeClassifier(MODEL_PATH+"/haarcascade_frontalface_default.xml")
    if haar_cascade.empty():
        print("Error: Failed to load cascade classifier.")
    else:
        print("Cascade classifier loaded successfully.")

    # Load images and resize them
    for image_file in image_files:
        image_path = os.path.join(image_folder, image_file)
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Read the image
        if img is not None:
            faces = haar_cascade.detectMultiScale(img, 1.1, 9)

            for (x, y, w, h) in faces:
            #cv2.rectangle(img, (x, y), (x+w, y+h), (0, 0, 255), 2)
                face = img[y:y + h, x:x + w]
            if face is None or img.size == 0:
                print("Error: Failed to load FACE.")
            else:
                print("FACE loaded successfully.")
            #cv2.imwrite('face.jpg', faces)
            img_resized = cv2.resize(face, target_size)  # Resize the image
            images.append(img_resized)  # Append the resized image
            print(f"Loaded image {image_file}")
        else:
            print(f"Failed to load image {image_file}.")

    return images

#### Perform data augmentation (only run for training)

In [ ]:
# TODO
cnt = 0;

train_image_map ={}
train_image_cnt={}
image_folder = TRAIN_DATA_PATH
label_file =  os.path.join(image_folder, 'labels.txt') ;
file1 = open(label_file, 'r')
labels = []

for line in file1:
  parts = line.split(' ')
  key = parts[1].strip()
  if key not in train_image_cnt:
    train_image_cnt[key] = 1  # Initialize the count for a new key
  else:
    train_image_cnt[key] += 1  # Increment the count for an existing key
  train_image_map[parts[0].strip()] = parts[1].strip()

if not os.path.exists(AUGMENTATION_DATA_PATH):
    os.makedirs(AUGMENTATION_DATA_PATH)

print("augmenting data")
#creating augmented images
for  img_orignal,label in train_image_map.items():

   # print("SIM  ", cnt);
    images = image_folder + '/' + img_orignal + '.jpg'
    #print(images)

    # Loading a sample image
    img = load_img(images)

    # Converting the input sample image to an array
    x = img_to_array(img)
    # Reshaping the input image
    x = x.reshape((1, ) + x.shape)

    i = 0


    for batch in datagen.flow(x, batch_size = 1,
                             save_to_dir =AUGMENTATION_DATA_PATH + "/",
                             save_prefix = label, save_format ='jpg'):

      i += 1
      if i > 5:
          break
    cnt +=1;

#label augmented images
files = os.listdir(AUGMENTATION_DATA_PATH)
fid = open(AUGMENTATION_DATA_PATH + '/augmented_labels.txt', 'w')

for f in files:
  if f.endswith('.jpg'):
    parts = f.split('_')
    name = f.split('.')[0]
    fid.write("{0} {1}\n".format(name,parts[0]))

fid.close()
print("data augmentation done")

augmenting data
data augmentation done


#### Read training/validation images and labels (only run for training)

In [ ]:
# TODO
#train_images, train_labels = load_images_and_labels(AUGMENTATION_DATA_PATH+"/augmented_labels.txt", AUGMENTATION_DATA_PATH)
#validation_images, validation_labels = load_images_and_labels(VALIDATION_DATA_PATH+"/labels.txt", VALIDATION_DATA_PATH)
#train_images, train_labels = cascade_load_images_and_labels(TRAIN_DATA_PATH+"/labels.txt", TRAIN_DATA_PATH)
train_images, train_labels = cascade_load_images_and_labels(AUGMENTATION_DATA_PATH+"/augmented_labels.txt", AUGMENTATION_DATA_PATH)
validation_images, validation_labels = cascade_load_images_and_labels(VALIDATION_DATA_PATH+"/labels.txt", VALIDATION_DATA_PATH)

Error: Failed to load cascade classifier.


error: OpenCV(4.8.0) /io/opencv/modules/objdetect/src/cascadedetect.cpp:1689: error: (-215:Assertion failed) !empty() in function 'detectMultiScale'


### Train and tune model (only run for training)

#### Perform data pre-processing

In [ ]:
# TODO
X_train = flatten_images(train_images)
X_validation = flatten_images(validation_images)

y_train = train_labels
y_validation = validation_labels


#### SVM Training

In [ ]:
# Initialize the StandardScaler
svm_sc = StandardScaler()
svm_sc.fit(X_train)
model_path = os.path.join(MODEL_PATH, 'svm_sc_model.joblib')
dump(svm_sc, model_path)
X_svm_train = sc.transform(X_train)
X_svm_validation = sc.transform(X_validation)

#using pca to reduce dimention
svm_pca = PCA(n_components=160)
svm_pca.fit(X_train,y_train)
# Save the model to the specified path
model_path = os.path.join(MODEL_PATH, 'svm_pca_model.joblib')
dump(svm_pca, model_path)
X_svm_train = svm_pca.transform(X_svm_train)
X_svm_validation = svm_pca.transform(X_svm_validation)

#using lda to reduce dimention
svm_lda = LinearDiscriminantAnalysis(n_components=26)
svm_lda.fit(X_svm_train, y_train)
model_path = os.path.join(MODEL_PATH, 'svm_lda_model.joblib')
dump(svm_lda, model_path)

X_svm_train = svm_lda.transform(X_svm_train)
X_svm_validation = svm_lda.transform(X_svm_validation)

svm_classifier = OneVsOneClassifier(SVC(kernel='rbf', C= 10, gamma = 0.001))

svm_classifier.fit(X_svm_train, y_train)

model_path = os.path.join(MODEL_PATH, 'svm_model.joblib')
dump(svm_classifier, model_path)


#### KNN Training

In [ ]:
# Initialize the StandardScaler
knn_sc = StandardScaler()
knn_sc.fit(X_train)
model_path = os.path.join(MODEL_PATH, 'knn_sc_model.joblib')
dump(knn_sc, model_path)
X_knn_train = knn_sc.transform(X_train)
X_knn_validation = knn_sc.transform(X_validation)

# Using PCA to reduce dimensionality
knn_pca = PCA(n_components=180)
knn_pca.fit(X_knn_train, y_train)
# Save the PCA model to the specified path
model_path = os.path.join(MODEL_PATH, 'knn_pca_model.joblib')
dump(knn_pca, model_path)
X_knn_train = knn_pca.transform(X_knn_train)
X_knn_validation = knn_pca.transform(X_knn_validation)

#using lda to reduce dimention
knn_lda = LinearDiscriminantAnalysis()
knn_lda.fit(X_knn_train, y_train)
model_path = os.path.join(MODEL_PATH, 'knn_lda_model.joblib')
dump(knn_lda, model_path)

X_knn_train = knn_lda.transform(X_knn_train)
X_knn_validation = knn_lda.transform(X_knn_validation)

# Initialize and train the KNN classifier
knn_classifier = KNeighborsClassifier(n_neighbors=3)
knn_classifier.fit(X_knn_train, y_train)

# Save the model to the specified path
model_path = os.path.join(MODEL_PATH, 'knn_model.joblib')
dump(knn_classifier, model_path)

#### Logistic Regression Training

In [ ]:
# Initialize the StandardScaler
logreg_sc = StandardScaler()
logreg_sc.fit(X_train)
model_path = os.path.join(MODEL_PATH, 'logreg_sc_model.joblib')
dump(logreg_sc, model_path)
X_logreg_train = logreg_sc.transform(X_train)
X_logreg_validation = logreg_sc.transform(X_validation)

# Using PCA to reduce dimensionality
logreg_pca = PCA(n_components=180)
logreg_pca.fit(X_logreg_train, y_train)
# Save the PCA model to the specified path
model_path = os.path.join(MODEL_PATH, 'logreg_pca_model.joblib')
dump(logreg_pca, model_path)
X_logreg_train = logreg_pca.transform(X_logreg_train)
X_logreg_validation = logreg_pca.transform(X_logreg_validation)

#using lda to reduce dimention
logreg_lda = LinearDiscriminantAnalysis()
logreg_lda.fit(X_logreg_train, y_train)
model_path = os.path.join(MODEL_PATH, 'logreg_lda_model.joblib')
dump(logreg_lda, model_path)

X_logreg_train = logreg_lda.transform(X_logreg_train)
X_logreg_validation = logreg_lda.transform(X_logreg_validation)


# Initialize and train the Logistic Regression classifier
logreg_classifier = LogisticRegression(C=0.1,max_iter=10000)
logreg_classifier.fit(X_logreg_train, y_train)

# Save the model to the specified path
model_path = os.path.join(MODEL_PATH, 'logreg_model.joblib')
dump(logreg_classifier, model_path)

#### Random Forest Training

In [ ]:
# Initialize the StandardScaler
rf_sc = StandardScaler()
rf_sc.fit(X_train)
model_path = os.path.join(MODEL_PATH, 'rf_sc_model.joblib')
dump(rf_sc, model_path)
X_rf_train = rf_sc.transform(X_train)
X_rf_validation = rf_sc.transform(X_validation)

# Using PCA to reduce dimensionality
rf_pca = PCA(n_components=180)
rf_pca.fit(X_rf_train, y_train)
# Save the PCA model to the specified path
model_path = os.path.join(MODEL_PATH, 'rf_pca_model.joblib')
dump(rf_pca, model_path)
X_rf_train = rf_pca.transform(X_rf_train)
X_rf_validation = rf_pca.transform(X_rf_validation)

#using lda to reduce dimention
rf_lda = LinearDiscriminantAnalysis()
rf_lda.fit(X_rf_train, y_train)
model_path = os.path.join(MODEL_PATH, 'rf_lda_model.joblib')
dump(rf_lda, model_path)

X_rf_train = rf_lda.transform(X_rf_train)
X_rf_validation = rf_lda.transform(X_rf_validation)

# Initialize and train the Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=500,max_depth=300)
rf_classifier.fit(X_rf_train, y_train)

# Save the model to the specified path
model_path = os.path.join(MODEL_PATH, 'rf_model.joblib')
dump(rf_classifier, model_path)

### Generate model performance reports (only run for training)

#### SVM Confusion Matrix and Accuracy

In [ ]:
# TODO
loaded_svm_classifier = load(os.path.join(MODEL_PATH, 'svm_model.joblib'))


# Evaluate the loaded model on training set
y_pred = loaded_svm_classifier.predict(X_svm_train)
cm = confusion_matrix(y_train, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=loaded_svm_classifier.classes_)
disp.plot()
plt.xticks(rotation=90, ha='right')
plt.show()

accuracy = accuracy_score(y_train, y_pred)
print("svm model accuracy on training set:", accuracy)

# Evaluate the loaded model on validation set
y_pred = loaded_svm_classifier.predict(X_svm_validation)
cm = confusion_matrix(y_validation, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=loaded_svm_classifier.classes_)
disp.plot()
plt.xticks(rotation=90, ha='right')
plt.show()

accuracy = accuracy_score(y_validation, y_pred)
print("svm model accuracy on validation set:", accuracy)

#### Random Forest Confusion Matrix and Accuracy

In [ ]:
# TODO
loaded_rf_classifier = load(os.path.join(MODEL_PATH, 'rf_model.joblib'))

# Evaluate the loaded model on training set
y_pred = loaded_rf_classifier.predict(X_rf_train)
cm = confusion_matrix(y_train, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=loaded_rf_classifier.classes_)
disp.plot()
plt.xticks(rotation=90, ha='right')
plt.show()

accuracy = accuracy_score(y_train, y_pred)
print("Random Forest model accuracy on training set:", accuracy)

# Evaluate the loaded model on validation set
y_pred = loaded_rf_classifier.predict(X_rf_validation)
cm = confusion_matrix(y_validation, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=loaded_rf_classifier.classes_)
disp.plot()
plt.xticks(rotation=90, ha='right')
plt.show()

accuracy = accuracy_score(y_validation, y_pred)
print("Random Forest model accuracy on validation set:", accuracy)

#### KNN Confusion Matrix and Accuracy

In [ ]:
# TODO
loaded_knn_classifier = load(os.path.join(MODEL_PATH, 'knn_model.joblib'))

# Evaluate the loaded model on training set
y_pred = loaded_knn_classifier.predict(X_knn_train)
cm = confusion_matrix(y_train, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=loaded_knn_classifier.classes_)
disp.plot()
plt.xticks(rotation=90, ha='right')
plt.show()

accuracy = accuracy_score(y_train, y_pred)
print("KNN model accuracy on training set:", accuracy)

# Evaluate the loaded model on validation set
y_pred = loaded_knn_classifier.predict(X_knn_validation)
cm = confusion_matrix(y_validation, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=loaded_knn_classifier.classes_)
disp.plot()
plt.xticks(rotation=90, ha='right')
plt.show()

accuracy = accuracy_score(y_validation, y_pred)
print("KNN model accuracy on validation set:", accuracy)

#### Logistic Regression Confusion Matrix and Accuracy

In [ ]:
# TODO
loaded_logreg_classifier = load(os.path.join(MODEL_PATH, 'logreg_model.joblib'))

# Evaluate the loaded model on training set
y_pred = loaded_logreg_classifier.predict(X_logreg_train)
cm = confusion_matrix(y_train, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=loaded_logreg_classifier.classes_)
disp.plot()
plt.xticks(rotation=90, ha='right')
plt.show()

accuracy = accuracy_score(y_train, y_pred)
print("Logistic Regression accuracy on training set:", accuracy)

# Evaluate the loaded model on validation set
y_pred = loaded_logreg_classifier.predict(X_logreg_validation)
cm = confusion_matrix(y_validation, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=loaded_logreg_classifier.classes_)
disp.plot()
plt.xticks(rotation=90, ha='right')
plt.show()

accuracy = accuracy_score(y_validation, y_pred)
print("Logistic Regression model accuracy on validation set:", accuracy)

#### Ensemble Confusion Matrix and Accuracy

In [ ]:
# TODO
loaded_svm_classifier = load(os.path.join(MODEL_PATH, 'svm_model.joblib'))
y_svm_pred = loaded_svm_classifier.predict(X_svm_train)

loaded_rf_classifier = load(os.path.join(MODEL_PATH, 'rf_model.joblib'))
y_rf_pred = loaded_rf_classifier.predict(X_rf_train)

loaded_knn_classifier = load(os.path.join(MODEL_PATH, 'knn_model.joblib'))
y_knn_pred = loaded_knn_classifier.predict(X_knn_train)

loaded_logreg_classifier = load(os.path.join(MODEL_PATH, 'logreg_model.joblib'))
y_logreg_pred = loaded_logreg_classifier.predict(X_logreg_train)

y_pred = hard_voting(y_svm_pred,y_rf_pred,y_knn_pred,y_logreg_pred)

cm = confusion_matrix(y_train, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=loaded_logreg_classifier.classes_)
disp.plot()
plt.xticks(rotation=90, ha='right')
plt.show()

accuracy = accuracy_score(y_train, y_pred)
print("Ensemble accuracy on training set:", accuracy)

# Evaluate the loaded model on validation set
y_svm_pred = loaded_svm_classifier.predict(X_svm_validation)

y_rf_pred = loaded_rf_classifier.predict(X_rf_validation)

y_knn_pred = loaded_knn_classifier.predict(X_knn_validation)

y_logreg_pred = loaded_logreg_classifier.predict(X_logreg_validation)

y_pred = hard_voting(y_svm_pred,y_rf_pred,y_knn_pred,y_logreg_pred)

cm = confusion_matrix(y_validation, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=loaded_logreg_classifier.classes_)
disp.plot()
plt.xticks(rotation=90, ha='right')
plt.show()

accuracy = accuracy_score(y_validation, y_pred)
print("Ensemble model accuracy on validation set:", accuracy)

### Read test images

In [32]:
# TODO
testing_images = cascade_load_images(TEST_DATA_PATH)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/SP24-CSEN240-2/Project/Test'

### Generate prediction file

In [30]:
# TODO Generate a group-x.txt file with predictions and save it to OUTPUT_PATH folder

X_testing = flatten_images(testing_images)



#scale
loaded_svm_sc = load(os.path.join(MODEL_PATH, 'svm_sc_model.joblib'))
X_svm_testing = loaded_svm_sc.transform(X_testing)
#pca
loaded_svm_pca = load(os.path.join(MODEL_PATH, 'svm_pca_model.joblib'))
X_svm_testing = loaded_svm_pca.transform(X_svm_testing)
#lda
loaded_svm_lda = load(os.path.join(MODEL_PATH, 'svm_lda_model.joblib'))
X_svm_testing = loaded_svm_lda.transform(X_svm_testing)
#svm prediction
loaded_svm_classifier = load(os.path.join(MODEL_PATH, 'svm_model.joblib'))
y_svm_pred = loaded_svm_classifier.predict(X_svm_testing)

#scale
loaded_rf_sc = load(os.path.join(MODEL_PATH, 'rf_sc_model.joblib'))
X_rf_testing = loaded_rf_sc.transform(X_testing)
#pca
loaded_rf_pca = load(os.path.join(MODEL_PATH, 'rf_pca_model.joblib'))
X_rf_testing = loaded_rf_pca.transform(X_rf_testing)
#lda
loaded_rf_lda = load(os.path.join(MODEL_PATH, 'rf_lda_model.joblib'))
X_rf_testing = loaded_rf_lda.transform(X_rf_testing)
#rf prediction
loaded_rf_classifier = load(os.path.join(MODEL_PATH, 'rf_model.joblib'))
y_rf_pred = loaded_rf_classifier.predict(X_rf_testing)

#scale
loaded_knn_sc = load(os.path.join(MODEL_PATH, 'knn_sc_model.joblib'))
X_knn_testing = loaded_knn_sc.transform(X_testing)
#pca
loaded_knn_pca = load(os.path.join(MODEL_PATH, 'knn_pca_model.joblib'))
X_knn_testing = loaded_knn_pca.transform(X_knn_testing)
#lda
loaded_knn_lda = load(os.path.join(MODEL_PATH, 'knn_lda_model.joblib'))
X_knn_testing = loaded_knn_lda.transform(X_knn_testing)
#knn prediction
loaded_knn_classifier = load(os.path.join(MODEL_PATH, 'knn_model.joblib'))
y_knn_pred = loaded_knn_classifier.predict(X_knn_testing)

#scale
loaded_logreg_sc = load(os.path.join(MODEL_PATH, 'logreg_sc_model.joblib'))
X_logreg_testing = loaded_logreg_sc.transform(X_testing)
#pca
loaded_logreg_pca = load(os.path.join(MODEL_PATH, 'logreg_pca_model.joblib'))
X_logreg_testing = loaded_logreg_pca.transform(X_logreg_testing)
#lda
loaded_logreg_lda = load(os.path.join(MODEL_PATH, 'logreg_lda_model.joblib'))
X_logreg_testing = loaded_logreg_lda.transform(X_logreg_testing)
#logreg prediction
loaded_logreg_classifier = load(os.path.join(MODEL_PATH, 'logreg_model.joblib'))
y_logreg_pred = loaded_logreg_classifier.predict(X_logreg_testing)

y_pred = hard_voting(y_svm_pred,y_rf_pred,y_knn_pred,y_logreg_pred)

write_predictions_to_file(y_pred,PREDICTION_FILE_PATH)

Predictions written to /content/drive/MyDrive/SP24-CSEN240-2/Project/Prediction/labels.txt


###Accuracy measurement script

In [31]:
from sklearn.metrics import accuracy_score

with open(TEST_LABEL_PATH, 'r') as f:
    y_test = [line.strip().split(' ')[1] for line in f.readlines()]

with open(PREDICTION_FILE_PATH, 'r') as f:
    y_pred = [line.strip().split(' ')[1] for line in f.readlines()]

score = accuracy_score(y_test, y_pred)
print('Accuracy score: %s'% score)

Accuracy score: 0.9281437125748503
